In [6]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [8]:
#get spam colletion using pandas
df_spam_collection = pd.read_csv('SpamCollection', sep='\t', names=['response', 'message'])

In [9]:
df_spam_collection.head()

,response,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
#view more using describe function

df_spam_collection.describe()

,response,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [11]:
#view response using groupby and describe method

df_spam_collection.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [12]:
#verify length of message and add it as new column feature
df_spam_collection['length'] = df_spam_collection['message'].apply(len)

In [13]:
#view first 5 with length
df_spam_collection.head()

,response,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [14]:
#define function to get rid of stop words
def message_text_process(mess):
    #check for punctuation
    no_punctuation = [char for char in mess if char not in string.punctuation]
    #form sentence
    no_punctuation = ''.join(no_punctuation)
    #now eliminate any stopwords
    return [word for word in no_punctuation.split() if word.lower() not in stopwords.words('english')]


In [15]:
#verify function
df_spam_collection['message'].head(5).apply(message_text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [16]:
#text processing using vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
#use bag of words by applying the message and fit the data (message) to it
bag_of_words_transformer = CountVectorizer(analyzer=message_text_process).fit(df_spam_collection['message'])

In [20]:
#print length of bag of words stored in vocabulary attribute
print(len(bag_of_words_transformer.vocabulary_))

11425


In [21]:
#store bag of words messages using transform method
message_bagofwords = bag_of_words_transformer.transform(df_spam_collection['message'])

In [23]:
#apply tdfidf transfromer 

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transform = TfidfTransformer().fit(message_bagofwords)

In [25]:
#print shape of tfidf 

message_tfidf = tfidf_transform.transform(message_bagofwords)
print(message_tfidf.shape)

(5572, 11425)


In [26]:
#chosse naives baise model
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,df_spam_collection['response'])

In [30]:
#check model for the preicted and executed value for the message 
message = df_spam_collection['message'][2]
bag_of_words_message = bag_of_words_transformer.transform([message])
tfidf = tfidf_transform.transform(bag_of_words_message)
print("predicted",spam_detect_model.predict(tfidf)[0])
print("expected",df_spam_collection.response[2])

predicted spam
expected spam
